To demonstrate the ``CPP().run()`` method, we load the ``DOM_GSEC`` example dataset (see [Breimann25a]_):

In [1]:
import aaanalysis as aa
aa.options["verbose"] = False
df_seq = aa.load_dataset(name="DOM_GSEC")
labels = df_seq["label"].to_list()
sf = aa.SequenceFeature()
df_parts = sf.get_df_parts(df_seq=df_seq)

You just need to provide ``df_parts`` to the ``CPP`` object and run the algorithm with its respective labels using the ``CPP().run()`` method:

In [2]:
cpp = aa.CPP(df_parts=df_parts)
# Create >500.000 feature and filter them down to 100 features 
df_feat = cpp.run(labels=labels)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (100, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_C_JMD_C-...)-QIAN880106,Conformation,α-helix,α-helix (middle),"Weights for ...owski, 1988)",0.387000,0.118000,0.118000,0.068000,0.080000,0.000000,0.000000,"27,28,29,30,31,32,33"
2,TMD_C_JMD_C-...)-CRAJ730103,Conformation,β-turn,β-turn,"Normalized f...t al., 1973)",0.377000,0.285000,-0.285000,0.164000,0.177000,0.000000,0.000000,"27,31"
3,TMD_C_JMD_C-...)-FAUJ880104,Shape,Side chain length,Steric parameter,"STERIMOL len...t al., 1988)",0.367000,0.263000,0.263000,0.161000,0.168000,0.000000,0.000000,"32,33"
4,TMD_C_JMD_C-...)-ONEK900101,Others,Unclassified (Others),ΔG values in peptides,"Delta G valu...Grado, 1990)",0.366000,0.111000,0.111000,0.070000,0.114000,0.000000,0.000000,"32,33"
5,TMD_C_JMD_C-...)-QIAN880107,Conformation,α-helix,α-helix (middle),"Weights for ...owski, 1988)",0.363000,0.162000,0.162000,0.091000,0.118000,0.000000,0.000000,"24,28,32,35"
6,TMD_C_JMD_C-...)-HUTJ700103,Energy,Entropy,Entropy,"Entropy of f...chens, 1970)",0.360000,0.187000,0.187000,0.115000,0.128000,0.000000,0.000000,"31,32,33,34,35"
7,TMD_C_JMD_C-...)-WOLS870103,Others,PC 4,Principal Co...ent 3 (Wold),"Principal pr...t al., 1987)",0.359000,0.159000,-0.159000,0.090000,0.130000,0.000000,0.000000,"27,28,29,30,31,32,33"
8,TMD_C_JMD_C-...)-CRAJ730103,Conformation,β-turn,β-turn,"Normalized f...t al., 1973)",0.352000,0.227000,-0.227000,0.150000,0.170000,0.000000,0.000000,"24,28,32"
9,TMD_C_JMD_C-...)-MUNV940102,Energy,Free energy (folding),Free energy (α-helix),"Free energy ...rrano, 1994)",0.350000,0.129000,-0.129000,0.079000,0.124000,0.000000,0.000000,"32,33"
10,TMD_C_JMD_C-...)-WOLS870103,Others,PC 4,Principal Co...ent 3 (Wold),"Principal pr...t al., 1987)",0.341000,0.214000,-0.214000,0.128000,0.177000,0.000000,0.000000,"31,32,33,34,35"


Adjust **Parts**, **Splits**, and **Scales** as follows:

In [13]:
df_parts = sf.get_df_parts(df_seq=df_seq, list_parts=["tmd_jmd"])
split_kws = sf.get_split_kws(split_types=["Segment"], n_split_min=1, n_split_max=5)
# Load one of the provided top scale datasets
df_scales = aa.load_scales(top60_n=38)  
# Create ~700 feature and filter them down to 19 features 
cpp = aa.CPP(df_parts=df_parts, split_kws=split_kws, df_scales=df_scales)
df_feat = cpp.run(labels=labels)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (19, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
3,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
4,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
5,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
6,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
7,TMD_JMD-Segm...)-FUKS010106,Composition,Membrane proteins (MPs),Proteins of ...philes (INT),"Interior com...ikawa, 2001)",0.277000,0.123000,0.123000,0.104000,0.127000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
8,TMD_JMD-Segm...)-WOLR790101,Polarity,Hydrophobici...surrounding),Hydration potential,"Hydrophobici...t al., 1979)",0.267000,0.105000,-0.105000,0.100000,0.113000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
9,TMD_JMD-Segm...)-CEDJ970105,Composition,AA composition,Nuclear proteins,"Composition ...t al., 1997)",0.263000,0.062000,0.062000,0.062000,0.069000,0.000000,0.000000,"21,22,23,24,...,37,38,39,40"
10,TMD_JMD-Segm...)-MITS020101,Polarity,Amphiphilicity,Amphiphilicity,"Amphiphilici...t al., 2002)",0.262000,0.073000,0.073000,0.071000,0.086000,0.000000,0.000000,"33,34,35,36,37,38,39,40"


The maximum number of final features can be adjusted using the ``n_filter`` (default=100) parameter. The actual number of features may be less, depending on: (a) the initial number of features generated (defined by the ``part-split-scale`` combinations), and (b) the strictness of both pre-filtering and filtering criteria.

In [14]:
# Create ~700 feature and filter them down to 10 features 
df_feat = cpp.run(labels=labels, n_filter=10)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (10, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
3,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
4,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
5,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
6,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
7,TMD_JMD-Segm...)-FUKS010106,Composition,Membrane proteins (MPs),Proteins of ...philes (INT),"Interior com...ikawa, 2001)",0.277000,0.123000,0.123000,0.104000,0.127000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
8,TMD_JMD-Segm...)-WOLR790101,Polarity,Hydrophobici...surrounding),Hydration potential,"Hydrophobici...t al., 1979)",0.267000,0.105000,-0.105000,0.100000,0.113000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
9,TMD_JMD-Segm...)-MIYS990104,Composition,MPs (anchor),Partition energy,"Optimized re...nigan, 1999)",0.243000,0.103000,0.103000,0.095000,0.126000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
10,TMD_JMD-Segm...)-ANDN920101,Structure-Activity,Backbone-dynamics (-CH),α-CH chemica...ne-dynamics),"alpha-CH che...t al., 1992)",0.229000,0.102000,-0.102000,0.097000,0.125000,0.000000,0.000000,"25,26,27,28,29,30,31,32"


In the initial CPP pre-filtering step, you can either set the number of retained features using ``n_pre_filter`` or define a percentage of initial features with ``pct_pre_filter`` (default with 5%). Additionally, adjust the maximum standard deviation allowed in the test dataset for each feature via ``max_std_test``:

In [15]:
# Pre-filtering by allowing 50% with 0.5 maximum std in the test set 
# Create ~700 feature and filter them down to 26 features
df_feat = cpp.run(labels=labels, pct_pre_filter=50, max_std_test=0.5)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (26, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
3,TMD_JMD-Segm...)-ONEK900101,Others,Unclassified (Others),ΔG values in peptides,"Delta G valu...Grado, 1990)",0.310000,0.041000,0.041000,0.028000,0.044000,0.000000,0.000000,"21,22,23,24,...,37,38,39,40"
4,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
5,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
6,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
7,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
8,TMD_JMD-Segm...)-FUKS010106,Composition,Membrane proteins (MPs),Proteins of ...philes (INT),"Interior com...ikawa, 2001)",0.277000,0.123000,0.123000,0.104000,0.127000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
9,TMD_JMD-Segm...)-WOLR790101,Polarity,Hydrophobici...surrounding),Hydration potential,"Hydrophobici...t al., 1979)",0.274000,0.052000,0.052000,0.034000,0.060000,0.000000,0.000000,"21,22,23,24,...,27,28,29,30"
10,TMD_JMD-Segm...)-WEBA780101,Others,Mutability,RF value,"RF value in ...Lacey, 1978)",0.268000,0.042000,0.042000,0.039000,0.046000,0.000000,0.000000,"1,2,3,4,5,6,...,17,18,19,20"


For the final CPP filtering step, you can use the following three parameters: ``max_overlap`` setting the allowed maximum positional overlap of similar features (the higher, the less strict), ``max_cor`` defining the allowed maximum Pearson correlation for scales of similar features (the higher, the less strict), and ``check_cat`` setting whether redundancy of scale categories should be considered or not (setting it to ``False`` will result in stricter filtering since features across all categories are compared): 

In [16]:
# Disable filtering by setting max_overlap and max_cor to 1
# Create ~700 feature and filter them down to 100 features
df_feat = cpp.run(labels=labels, max_overlap=1, max_cor=1)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (100, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
3,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.304000,0.069000,0.069000,0.051000,0.073000,0.000000,0.000000,"27,28,29,30,...,37,38,39,40"
4,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
5,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
6,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.292000,0.058000,0.058000,0.045000,0.054000,0.000000,0.000000,"21,22,23,24,...,37,38,39,40"
7,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
8,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.291000,0.127000,0.127000,0.097000,0.121000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
9,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
10,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.288000,0.164000,0.164000,0.135000,0.145000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"


In [17]:
# Perform stricter filtering by setting check_cat=False
# Create ~700 feature and filter them down to 11 features
df_feat = cpp.run(labels=labels, check_cat=False)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (11, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"
3,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
4,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
5,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
6,TMD_JMD-Segm...)-CEDJ970105,Composition,AA composition,Nuclear proteins,"Composition ...t al., 1997)",0.263000,0.062000,0.062000,0.062000,0.069000,0.000000,0.000000,"21,22,23,24,...,37,38,39,40"
7,TMD_JMD-Segm...)-MITS020101,Polarity,Amphiphilicity,Amphiphilicity,"Amphiphilici...t al., 2002)",0.262000,0.073000,0.073000,0.071000,0.086000,0.000000,0.000000,"33,34,35,36,37,38,39,40"
8,TMD_JMD-Segm...)-SIMZ760101,Polarity,Hydrophobicity,Transfer fre...) to outside,Transfer fre...arton (1982),0.259000,0.064000,-0.064000,0.069000,0.072000,0.000000,0.000000,"1,2,3,4,5,6,...,17,18,19,20"
9,TMD_JMD-Segm...)-ANDN920101,Structure-Activity,Backbone-dynamics (-CH),α-CH chemica...ne-dynamics),"alpha-CH che...t al., 1992)",0.229000,0.102000,-0.102000,0.097000,0.125000,0.000000,0.000000,"25,26,27,28,29,30,31,32"
10,TMD_JMD-Segm...)-YUTK870103,Energy,Free energy (unfolding),Free energy (unfolding),"Activation G...t al., 1987)",0.201000,0.084000,-0.084000,0.115000,0.118000,0.000000,0.000000,"31,32,33,34,...,37,38,39,40"


The residue positions can be adjusted using the ``start``, ``tmd_len``, ``jmd_n_len``, and ``jmd_c_len`` parameters:

In [18]:
# Shift positions by 10 residues
df_feat = cpp.run(labels=labels, start=11)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (19, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"35,36,37,38,39,40,41,42"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"41,42,43,44,...,47,48,49,50"
3,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"35,36,37,38,39,40,41,42"
4,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"43,44,45,46,47,48,49,50"
5,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"43,44,45,46,47,48,49,50"
6,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"43,44,45,46,47,48,49,50"
7,TMD_JMD-Segm...)-FUKS010106,Composition,Membrane proteins (MPs),Proteins of ...philes (INT),"Interior com...ikawa, 2001)",0.277000,0.123000,0.123000,0.104000,0.127000,0.000000,0.000000,"35,36,37,38,39,40,41,42"
8,TMD_JMD-Segm...)-WOLR790101,Polarity,Hydrophobici...surrounding),Hydration potential,"Hydrophobici...t al., 1979)",0.267000,0.105000,-0.105000,0.100000,0.113000,0.000000,0.000000,"41,42,43,44,...,47,48,49,50"
9,TMD_JMD-Segm...)-CEDJ970105,Composition,AA composition,Nuclear proteins,"Composition ...t al., 1997)",0.263000,0.062000,0.062000,0.062000,0.069000,0.000000,0.000000,"31,32,33,34,...,47,48,49,50"
10,TMD_JMD-Segm...)-MITS020101,Polarity,Amphiphilicity,Amphiphilicity,"Amphiphilici...t al., 2002)",0.262000,0.073000,0.073000,0.071000,0.086000,0.000000,0.000000,"43,44,45,46,47,48,49,50"


In [19]:
# Increase TMD length from 20 to 50
df_feat = cpp.run(labels=labels, tmd_len=50)
aa.display_df(df_feat, n_rows=10, show_shape=True)

DataFrame shape: (19, 13)


,feature,category,subcategory,scale_name,scale_description,abs_auc,abs_mean_dif,mean_dif,std_test,std_ref,p_val_mann_whitney,p_val_fdr_bh,positions
1,TMD_JMD-Segm...)-ROBB760113,Conformation,β-turn,β-turn,"Information ...uzuki, 1976)",0.316000,0.137000,-0.137000,0.102000,0.108000,0.000000,0.000000,"43,44,45,46,...,53,54,55,56"
2,TMD_JMD-Segm...)-ZIMJ680104,Energy,Isoelectric point,Isoelectric point,"Isoelectric ...t al., 1968)",0.312000,0.099000,0.099000,0.069000,0.095000,0.000000,0.000000,"53,54,55,56,...,67,68,69,70"
3,TMD_JMD-Segm...)-KANM800103,Conformation,α-helix,α-helix,"Average rela...Tsong, 1980)",0.297000,0.086000,0.086000,0.077000,0.068000,0.000000,0.000000,"43,44,45,46,...,53,54,55,56"
4,TMD_JMD-Segm...)-LINS030104,ASA/Volume,Accessible s...e area (ASA),ASA (folded protein),"Total median...t al., 2003)",0.295000,0.141000,0.141000,0.115000,0.130000,0.000000,0.000000,"57,58,59,60,...,67,68,69,70"
5,TMD_JMD-Segm...)-JANJ780102,ASA/Volume,Buried,Buried,"Percentage o...t al., 1978)",0.291000,0.130000,-0.130000,0.099000,0.124000,0.000000,0.000000,"57,58,59,60,...,67,68,69,70"
6,TMD_JMD-Segm...)-ZIMJ680103,Polarity,Hydrophilicity,Polarity (hydrophilicity),"Polarity (Zi...t al., 1968)",0.289000,0.178000,0.178000,0.159000,0.163000,0.000000,0.000000,"57,58,59,60,...,67,68,69,70"
7,TMD_JMD-Segm...)-FUKS010106,Composition,Membrane proteins (MPs),Proteins of ...philes (INT),"Interior com...ikawa, 2001)",0.277000,0.123000,0.123000,0.104000,0.127000,0.000000,0.000000,"43,44,45,46,...,53,54,55,56"
8,TMD_JMD-Segm...)-WOLR790101,Polarity,Hydrophobici...surrounding),Hydration potential,"Hydrophobici...t al., 1979)",0.267000,0.105000,-0.105000,0.100000,0.113000,0.000000,0.000000,"53,54,55,56,...,67,68,69,70"
9,TMD_JMD-Segm...)-CEDJ970105,Composition,AA composition,Nuclear proteins,"Composition ...t al., 1997)",0.263000,0.062000,0.062000,0.062000,0.069000,0.000000,0.000000,"36,37,38,39,...,67,68,69,70"
10,TMD_JMD-Segm...)-MITS020101,Polarity,Amphiphilicity,Amphiphilicity,"Amphiphilici...t al., 2002)",0.262000,0.073000,0.073000,0.071000,0.086000,0.000000,0.000000,"57,58,59,60,...,67,68,69,70"


Multiprocessing can be enabled by using the ``n_jobs`` parameter, which is set to the maximum if ``n_jobs=None``. However, this is only recommend for more than ~1000 features per core due to potential process management overhead.  

In [20]:
import time

# Run without multiprocessing
time_start = time.time()
df_feat = cpp.run(labels=labels, n_jobs=1)
time_no_mp = round(time.time() - time_start, 2)
print(f"Time without multiprocessing: {time_no_mp} seconds")

# Run with multiprocessing
time_start = time.time()
df_feat = cpp.run(labels=labels, n_jobs=None)
time_mp = round(time.time() - time_start, 2)
print(f"Time with multiprocessing. {time_mp} seconds")

Time without multiprocessing: 6.9 seconds
Time with multiprocessing. 10.84 seconds
